# `CLOUD.jl` - 2D linear advection example - energy-conservative scheme in collapsed coordinates

In [1]:
using CLOUD, OrdinaryDiffEq

┌ Info: Precompiling CLOUD [fb992021-99c7-4c2d-a14b-5e48ac4045b2]
└ @ Base loading.jl:1423


Define the physical problem

In [2]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

Set discretization parameters

In [3]:
M = 2
p = 8
p_map = 8
form = WeakConservationForm(mapping_form=SkewSymmetricMapping(),
    inviscid_numerical_flux=LaxFriedrichsNumericalFlux(0.0))
strategy = Lazy()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.01*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

Set up a `CollapsedSEM` scheme of degree `p` with `M` edges in each direction

In [4]:
reference_approximation =ReferenceApproximation(
    CollapsedSEM(p), Tri(),
    mapping_degree=p_map, N_plot=50)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [5]:
ode_problem = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:           501ms / 2738.2%          57.3MiB / 16773.7%   

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual           192k    11.6s   84.5%  60.2μs   7.09GiB   75.4%  38.6KiB
   volume terms          192k    8.90s   64.9%  46.2μs   5.89GiB   62.7%  32.1KiB
   mass matrix solve     192k    1.64s   11.9%  8.51μs    270MiB    2.8%  1.44KiB
   facet terms           192k    649ms    4.7%  3.37μs    620MiB    6.4%  3.30KiB
 extrap solution         192k    1.22s    8.9%  6.32μs    682MiB    7.1%  3.63KiB
 eval flux               192k    440ms    3.2%  2.29μs    596MiB    6.2%  3.17KiB
 eval num flux 

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [6]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.0030761191433130676
Conservation (initial/final/diff):


LoadError: DimensionMismatch("second dimension of left factor, 81, does not match first dimension of right factor, 36")